# Cell BLAST tutorial

In [1]:
import time
import warnings
import anndata
import Cell_BLAST as cb

warnings.filterwarnings("ignore")
cb.config.N_JOBS = 4
cb.config.RANDOM_SEED = 0

## Preparing database

In this tutorial, we demonstrate how to perform Cell BLAST based on DIRECTi models.

Again, we use the human pancreatic islet dataset [Baron_human](http://download.gao-lab.org/Cell_BLAST/tutorial/Baron_human.h5ad) as an example.

In [2]:
baron_human = anndata.read_h5ad("Baron_human.h5ad")

In [3]:
%%capture
axes = cb.data.find_variable_genes(baron_human, grouping="donor")

Cell BLAST uses multiple models to increase specificity.

Here we first train 4 DIRECTi models, each with a different random seed.

> Please refer to the accompanying [DIRECTi](DIRECTi.ipynb) notebook for more detailed introduction to model training.

In [4]:
%%capture
start_time=time.time()
models = []
for i in range(4):
    models.append(cb.directi.fit_DIRECTi(
        baron_human, genes=baron_human.var.query("variable_genes").index,
        latent_dim=10, cat_dim=20, random_seed=i
    ))

[INFO] Cell BLAST: Using model path: /tmp/tmp3k89_jx1
[INFO] Cell BLAST: Using model path: /tmp/tmpm1gqyon4
[INFO] Cell BLAST: Using model path: /tmp/tmpj6zf6z9g
[INFO] Cell BLAST: Using model path: /tmp/tmpglx_f5fl


In [5]:
print("Time elapsed: %.1fs" % (time.time() - start_time))

Time elapsed: 768.9s


Then we build a Cell BLAST "database" by feeding our previously trained models and the reference dataset.

In [6]:
blast = cb.blast.BLAST(models, baron_human)

[INFO] Cell BLAST: Projecting to latent space...
[INFO] Cell BLAST: Fitting nearest neighbor trees...
[INFO] Cell BLAST: Sampling from posteriors...
/home/lurw/anaconda3/envs/cbp/lib/python3.9/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/home/lurw/anaconda3/envs/cbp/lib/python3.9/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/home/lurw/anaconda3/envs/cbp/lib/python3.9/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/home/lurw/anaconda3/envs/cbp/lib/python3.9/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not sup

Like DIRECTi models, [BLAST](modules/Cell_BLAST.blast.rst#Cell_BLAST.blast.BLAST) objects can be easily saved and loaded.

In [7]:
blast.save("./baron_human_blast")
del blast
blast = cb.blast.BLAST.load("./baron_human_blast")

[INFO] Cell BLAST: Using model path: ./baron_human_blast/model_0
[INFO] Cell BLAST: Using model path: ./baron_human_blast/model_1
[INFO] Cell BLAST: Using model path: ./baron_human_blast/model_2
[INFO] Cell BLAST: Using model path: ./baron_human_blast/model_3
[INFO] Cell BLAST: Fitting nearest neighbor trees...


## Querying

We load another human pancreatic islet dataset [Lawlor](http://download.gao-lab.org/Cell_BLAST/tutorial/Lawlor.h5ad) to demonstrate the querying process.

Note that we do **NOT** perform data normalization or gene subsetting here. These should be internally handled by the BLAST object later in querying.

In [8]:
lawlor = anndata.read_h5ad("Lawlor.h5ad")

To query the database, we first use the [query()](modules/Cell_BLAST.blast.rst#Cell_BLAST.blast.BLAST.query) method to obtain initial hits in the reference database. This is done by efficient Euclidean distance based nearest neighbor search in the latent space. Nearest neighbors in the latent space of each model will be merged. Though highly efficient, latent space Euclidean distance is not the best metric to determine cell-cell similarity. To increase accuracy and specificity, we also compute posterior distribution distances as well as empirical p-values for these nearest neighbors.

In [9]:
start_time = time.time()
lawlor_hits = blast.query(lawlor)
print("Time per query: %.1fms" % (
    (time.time() - start_time) * 1000 / lawlor.shape[0]
))

[INFO] Cell BLAST: Projecting to latent space...
[WARNING] Cell BLAST: 21 out of 769 variables are not found, will be set to zero!
[INFO] Cell BLAST: 'ADGRF5', 'ADGRL4', 'CD24', 'CEMIP', 'CXCL8', 'H19', 'HLA.B', 'HLA.DPA1', 'HLA.DPB1', 'HLA.DRA', 'HLA.DRB1', 'MIR4435.2HG', 'NEURL3', 'PCAT19', 'PECAM1', 'PRSS2', 'SMIM24', 'TPSB2', 'UCA1', 'UNQ6494', 'pk'
[WARNING] Cell BLAST: 21 out of 769 variables are not found, will be set to zero!
[INFO] Cell BLAST: 'ADGRF5', 'ADGRL4', 'CD24', 'CEMIP', 'CXCL8', 'H19', 'HLA.B', 'HLA.DPA1', 'HLA.DPB1', 'HLA.DRA', 'HLA.DRB1', 'MIR4435.2HG', 'NEURL3', 'PCAT19', 'PECAM1', 'PRSS2', 'SMIM24', 'TPSB2', 'UCA1', 'UNQ6494', 'pk'
[WARNING] Cell BLAST: 21 out of 769 variables are not found, will be set to zero!
[INFO] Cell BLAST: 'ADGRF5', 'ADGRL4', 'CD24', 'CEMIP', 'CXCL8', 'H19', 'HLA.B', 'HLA.DPA1', 'HLA.DPB1', 'HLA.DRA', 'HLA.DRB1', 'MIR4435.2HG', 'NEURL3', 'PCAT19', 'PECAM1', 'PRSS2', 'SMIM24', 'TPSB2', 'UCA1', 'UNQ6494', 'pk'
[WARNING] Cell BLAST: 21 out o

Time per query: 72.5ms


Then we use [reconcile_models()](modules/Cell_BLAST.blast.rst#Cell_BLAST.blast.Hits.reconcile_models) to pool together informarion from multiple models and [filter()](modules/Cell_BLAST.blast.rst#Cell_BLAST.blast.Hits.filter) the initial hits to obtain significant hits.

In [10]:
lawlor_hits = lawlor_hits.reconcile_models().filter(by="pval", cutoff=0.05)

Optionally, we may use the [to_data_frames()](modules/Cell_BLAST.blast.rst#Cell_BLAST.blast.Hits.to_data_frames) method to extract detailed information about the query hits.

The return value is a python dict, with query cell names as keys and meta table of query hits as values.

In [11]:
hits_dict = lawlor_hits[0:5].to_data_frames()
hits_dict.keys()

odict_keys(['1st-61_S27', '1st-C11_S58', '1st-C13_S19', '1st-C15_S3', '1st-C18_S51'])

In [12]:
hits_dict["1st-61_S27"]

donor cell_type1 library      organism  \
cell_id                                                              
human1_lib1.final_cell_0078     1       beta    lib1  Homo sapiens   
human1_lib1.final_cell_0170     1       beta    lib1  Homo sapiens   
human1_lib2.final_cell_0240     1       beta    lib2  Homo sapiens   
human1_lib2.final_cell_0274     1       beta    lib2  Homo sapiens   
human1_lib3.final_cell_0339     1       beta    lib3  Homo sapiens   
human2_lib1.final_cell_0254     2       beta    lib1  Homo sapiens   
human2_lib2.final_cell_0061     2       beta    lib2  Homo sapiens   
human2_lib2.final_cell_0398     2       beta    lib2  Homo sapiens   
human2_lib3.final_cell_0107     2       beta    lib3  Homo sapiens   
human2_lib3.final_cell_0215     2       beta    lib3  Homo sapiens   
human2_lib3.final_cell_0543     2       beta    lib3  Homo sapiens   
human3_lib3.final_cell_0356     3       beta    lib3  Homo sapiens   
human3_lib4.final_cell_0739     3       beta    lib4  Homo sapiens   
human3_lib4.final_cell_0810     3       beta    lib4  Homo sapiens   
human4_lib3.final_cell_0117     4       beta    lib3  Homo sapiens   
human4_lib3.final_cell_0458     4       beta    lib3  Homo sapiens   

                            dataset_name platform     organ data_type  \
cell_id                                                                 
human1_lib1.final_cell_0078  Baron_human   inDrop  Pancreas       raw   
human1_lib1.final_cell_0170  Baron_human   inDrop  Pancreas       raw   
human1_lib2.final_cell_0240  Baron_human   inDrop  Pancreas       raw   
human1_lib2.final_cell_0274  Baron_human   inDrop  Pancreas       raw   
human1_lib3.final_cell_0339  Baron_human   inDrop  Pancreas       raw   
human2_lib1.final_cell_0254  Baron_human   inDrop  Pancreas       raw   
human2_lib2.final_cell_0061  Baron_human   inDrop  Pancreas       raw   
human2_lib2.final_cell_0398  Baron_human   inDrop  Pancreas       raw   
human2_lib3.final_cell_0107  Baron_human   inDrop  Pancreas       raw   
human2_lib3.final_cell_0215  Baron_human   inDrop  Pancreas       raw   
human2_lib3.final_cell_0543  Baron_human   inDrop  Pancreas       raw   
human3_lib3.final_cell_0356  Baron_human   inDrop  Pancreas       raw   
human3_lib4.final_cell_0739  Baron_human   inDrop  Pancreas       raw   
human3_lib4.final_cell_0810  Baron_human   inDrop  Pancreas       raw   
human4_lib3.final_cell_0117  Baron_human   inDrop  Pancreas       raw   
human4_lib3.final_cell_0458  Baron_human   inDrop  Pancreas       raw   

                                cell_ontology_class cell_ontology_id  n_genes  \
cell_id                                                                         
human1_lib1.final_cell_0078  type B pancreatic cell       CL:0000169     2898   
human1_lib1.final_cell_0170  type B pancreatic cell       CL:0000169     2478   
human1_lib2.final_cell_0240  type B pancreatic cell       CL:0000169     2096   
human1_lib2.final_cell_0274  type B pancreatic cell       CL:0000169     2030   
human1_lib3.final_cell_0339  type B pancreatic cell       CL:0000169     1981   
human2_lib1.final_cell_0254  type B pancreatic cell       CL:0000169     2522   
human2_lib2.final_cell_0061  type B pancreatic cell       CL:0000169     2963   
human2_lib2.final_cell_0398  type B pancreatic cell       CL:0000169     1617   
human2_lib3.final_cell_0107  type B pancreatic cell       CL:0000169     2293   
human2_lib3.final_cell_0215  type B pancreatic cell       CL:0000169     2104   
human2_lib3.final_cell_0543  type B pancreatic cell       CL:0000169      919   
human3_lib3.final_cell_0356  type B pancreatic cell       CL:0000169     1961   
human3_lib4.final_cell_0739  type B pancreatic cell       CL:0000169     1090   
human3_lib4.final_cell_0810  type B pancreatic cell       CL:0000169     1160   
human4_lib3.final_cell_0117  type B pancreatic cell       CL:0000169     2072   
human4_lib3.final_cell_0458  type B pancreatic cell       CL:0000169     19

Finally, we can use the [annotate()](modules/Cell_BLAST.blast.rst#Cell_BLAST.blast.Hits.annotate) method to obtain cell type predictions.

In [13]:
lawlor_predictions = lawlor_hits.annotate("cell_ontology_class")

For the "Lawlor" dataset, we also have author provided "ground truth" cell type annotations.

By comparing with the "ground truth", we see that the predictions are quite accurate.

In [14]:
fig = cb.blast.sankey(
    lawlor.obs["cell_ontology_class"].values,
    lawlor_predictions.values.ravel(),
    title="Lawlor to Baron_human", tint_cutoff=2
)